### Notebook to calculate NPMRDS level of travel-time reliability (LOTTR) metric for trucks

In [ ]:
import pandas as pd
import datetime
from dbfread import DBF

In [ ]:
# Enums (symbolic constants) for NHPP Truck Periods:
TRUCK_NONE         = 0
TRUCK_WEEKDAY_6_10 = 1
TRUCK_WEEKDAY_10_4 = 2
TRUCK_WEEKDAY_4_8  = 3
TRUCK_WEEKEND_6_8  = 4
TRUCK_ALLDAYS_8_6  = 5

In [ ]:
# The truck periods are:
#     1. weekdays 6 AM to 10 AM
#     2. weekdays 10 AM to 4 PM
#     3. weekdays 4 PM to 8 PM
#     4. weekends 6 AM to 8 PM
#     5. all days 8 PM to 6 AM
#
# NOTE: In the input data, 'hours' are given using the 24-hour (NOT 12-hour) clock.
#       For example, 4 PM == 16 hours
#
def get_truck_period(dow, hour):
	retval = TRUCK_NONE
	if (dow == 5 or dow == 6):
		if hour >=6 and hour < 20:
			retval = TRUCK_WEEKEND_6_8
		else:
			retval = TRUCK_ALLDAYS_8_6
	elif hour >=6 and hour < 10:
		retval = TRUCK_WEEKDAY_6_10
	elif hour >= 10 and hour < 16:
		retval = TRUCK_WEEKDAY_10_4
	elif hour >= 16 and hour < 20:
		retval = TRUCK_WEEKDAY_4_8
	else:
		retval = TRUCK_ALLDAYS_8_6
	# endif
	return retval
# end_def get_truck_period()

In [ ]:
ritis_data_csv =   r'//lilliput/bkrepp/jupyter_notebooks/npmrds-tt-reliability/data/npmrds-trucks-2019.csv'

In [ ]:
ritis_df = pd.read_csv(ritis_data_csv)

In [ ]:
ritis_df.head(10)

In [ ]:
ritis_df['datepart'] = ritis_df.apply(lambda row : row['measurement_tstamp'].split(' ')[0], axis=1)

In [ ]:
ritis_df['timepart'] = ritis_df.apply(lambda row : row['measurement_tstamp'].split(' ')[1], axis=1)

In [ ]:
ritis_df['mo'] = ritis_df.apply(lambda row : int(row['datepart'].split('-')[1]), axis=1)

In [ ]:
ritis_df['dy'] = ritis_df.apply(lambda row : int(row['datepart'].split('-')[2]), axis=1)

In [ ]:
ritis_df.head(10)

In [ ]:
# Assumes all records are for 2019.
ritis_df['dow'] = ritis_df.apply(lambda row : datetime.date(2019, row['mo'], row['dy']).weekday(), axis=1)

In [ ]:
ritis_df['hour'] = ritis_df.apply(lambda row: int(row['timepart'].split(':')[0]), axis=1)

In [ ]:
ritis_df['truck_period'] = ritis_df.apply(lambda row : get_truck_period(row['dow'], row['hour']), axis=1)

### Calculate level of travel-time reliability metric for all TMCs for Truck time period 1

In [ ]:
period_1_df = ritis_df.copy(deep=True)
period_1_df = period_1_df[period_1_df['truck_period'] == TRUCK_WEEKDAY_6_10]

g50 = period_1_df.groupby('tmc_code')['travel_time_seconds'].quantile(q=0.5)
period_1_g50_df = g50.to_frame()
period_1_g50_df = period_1_g50_df.rename(columns={'travel_time_seconds' : "p1_tt_secs_50pct"})

g95 = period_1_df.groupby('tmc_code')['travel_time_seconds'].quantile(q=0.95)
period_1_g95_df = g95.to_frame()
period_1_g95_df = period_1_g95_df.rename(columns={'travel_time_seconds' : "p1_tt_secs_95pct"})

period_1_stats_df = period_1_g50_df.merge(right=period_1_g95_df, left_on='tmc_code', right_on='tmc_code')
period_1_stats_df['p1_lottr'] = round(period_1_stats_df['p1_tt_secs_95pct'] / period_1_stats_df['p1_tt_secs_50pct'], 2)

In [ ]:
period_1_stats_df.head(10)

### Calculate level of travel-time reliability metric for all TMCs for Truck time period 2

In [ ]:
period_2_df = ritis_df.copy(deep=True)
period_2_df = period_2_df[period_2_df['truck_period'] == TRUCK_WEEKDAY_10_4]

g50 = period_2_df.groupby('tmc_code')['travel_time_seconds'].quantile(q=0.5)
period_2_g50_df = g50.to_frame()
period_2_g50_df = period_2_g50_df.rename(columns={'travel_time_seconds' : "p2_tt_secs_50pct"})

g95 = period_2_df.groupby('tmc_code')['travel_time_seconds'].quantile(q=0.95)
period_2_g95_df = g95.to_frame()
period_2_g95_df = period_2_g95_df.rename(columns={'travel_time_seconds' : "p2_tt_secs_95pct"})

period_2_stats_df = period_2_g50_df.merge(right=period_2_g95_df, left_on='tmc_code', right_on='tmc_code')
period_2_stats_df['p2_lottr'] = round(period_2_stats_df['p2_tt_secs_95pct'] / period_2_stats_df['p2_tt_secs_50pct'], 2)

In [ ]:
period_2_stats_df.head(10)

### Calculate level of travel-time reliability metrci for all TMCs for Truck time period 3

In [ ]:
period_3_df = ritis_df.copy(deep=True)
period_3_df = period_3_df[period_3_df['truck_period'] == TRUCK_WEEKDAY_4_8]

g50 = period_3_df.groupby('tmc_code')['travel_time_seconds'].quantile(q=0.5)
period_3_g50_df = g50.to_frame()
period_3_g50_df = period_3_g50_df.rename(columns={'travel_time_seconds' : "p3_tt_secs_50pct"})

g95 = period_3_df.groupby('tmc_code')['travel_time_seconds'].quantile(q=0.95)
period_3_g95_df = g95.to_frame()
period_3_g95_df = period_3_g95_df.rename(columns={'travel_time_seconds' : "p3_tt_secs_95pct"})

period_3_stats_df = period_3_g50_df.merge(right=period_3_g95_df, left_on='tmc_code', right_on='tmc_code')
period_3_stats_df['p3_lottr'] = round(period_3_stats_df['p3_tt_secs_95pct'] / period_3_stats_df['p3_tt_secs_50pct'], 2)

In [ ]:
period_3_stats_df.head(10)

### Calculate level of travel-time reliabilty metric for all TMCs for Truck time period 4

In [ ]:
period_4_df = ritis_df.copy(deep=True)
period_4_df = period_4_df[period_4_df['truck_period'] == TRUCK_WEEKEND_6_8 ]

g50 = period_4_df.groupby('tmc_code')['travel_time_seconds'].quantile(q=0.5)
period_4_g50_df = g50.to_frame()
period_4_g50_df = period_4_g50_df.rename(columns={'travel_time_seconds' : "p4_tt_secs_50pct"})

g95 = period_4_df.groupby('tmc_code')['travel_time_seconds'].quantile(q=0.95)
period_4_g95_df = g95.to_frame()
period_4_g95_df = period_4_g95_df.rename(columns={'travel_time_seconds' : "p4_tt_secs_95pct"})

period_4_stats_df = period_4_g50_df.merge(right=period_4_g95_df, left_on='tmc_code', right_on='tmc_code')
period_4_stats_df['p4_lottr'] = round(period_4_stats_df['p4_tt_secs_95pct'] / period_4_stats_df['p4_tt_secs_50pct'], 2)

In [ ]:
period_4_stats_df.head(10)

### Calculate level of travel-time reliability metric for all TMCs for Truck time period 5

In [ ]:
period_5_df = ritis_df.copy(deep=True)
period_5_df = period_5_df[period_5_df['truck_period'] == TRUCK_ALLDAYS_8_6 ]

g50 = period_5_df.groupby('tmc_code')['travel_time_seconds'].quantile(q=0.5)
period_5_g50_df = g50.to_frame()
period_5_g50_df = period_5_g50_df.rename(columns={'travel_time_seconds' : "p5_tt_secs_50pct"})

g95 = period_5_df.groupby('tmc_code')['travel_time_seconds'].quantile(q=0.95)
period_5_g95_df = g95.to_frame()
period_5_g95_df = period_5_g95_df.rename(columns={'travel_time_seconds' : "p5_tt_secs_95pct"})

period_5_stats_df = period_5_g50_df.merge(right=period_5_g95_df, left_on='tmc_code', right_on='tmc_code')
period_5_stats_df['p5_lottr'] = round(period_5_stats_df['p5_tt_secs_95pct'] / period_5_stats_df['p5_tt_secs_50pct'], 2)

In [ ]:
period_5_stats_df.head(10)

### Join time-period specific dataframes into a single dataframe

In [ ]:
j1 = period_1_stats_df.merge(right=period_2_stats_df, left_on='tmc_code', right_on='tmc_code')
j2 = j1.merge(right=period_3_stats_df, left_on='tmc_code', right_on='tmc_code')
j3 = j2.merge(right=period_4_stats_df, left_on='tmc_code', right_on='tmc_code')
j4 = j3.merge(right=period_5_stats_df, left_on='tmc_code', right_on='tmc_code')

In [ ]:
j4 = j4.drop(columns=['p1_tt_secs_50pct', 'p1_tt_secs_95pct', 'p2_tt_secs_50pct', 'p2_tt_secs_95pct',
                      'p3_tt_secs_50pct', 'p3_tt_secs_95pct', 'p4_tt_secs_50pct', 'p4_tt_secs_95pct',
                      'p5_tt_secs_50pct', 'p5_tt_secs_95pct'])

In [ ]:
j4.head(10)

### Prepare final dataframe with LOTTR metric for each TMC for output

In [ ]:
final_df = j4.rename(columns={'p1_lottr' : 'LOTTR Weekday 6 AM -10 AM',
                              'p2_lottr' : 'LOTTR Weekday 10 AM - 4 PM',
                              'p3_lottr' : 'LOTTR Weekday 4 PM - 8 PM',
                              'p4_lottr' : 'LOTTR Weekend 6 AM - 8 PM',
                              'p5_lottr' : 'LOTTR All days 8 PM - 6 AM'})

In [ ]:
final_df.head(10)

### Export dataframe with LOTTR metric for each TMC to a CSV file

In [ ]:
output_csv_fn = r'//lilliput/bkrepp/jupyter_notebooks/npmrds-tt-reliability/lottr_trucks.csv'

In [ ]:
final_df.to_csv(output_csv_fn)

###  Calculate the single regional LOTTR metric for trucks

In [ ]:
# Prep dataframe to be joined to TMC attribute table data. 
# Add column for 'average volume'
j4['max_lottr'] = j4.apply(lambda row: max(row['p1_lottr'], row['p2_lottr'], row['p3_lottr'], 
                                           row['p4_lottr'], row['p5_lottr']), axis=1)

In [ ]:
# DBF file with attribute table for NPMPRDS shapefile
tmc_attr_table_fn = r'//lilliput/bkrepp/jupyter_notebooks/npmrds-tt-reliability/shapefile/BRMPO_NPMRDS_TMCS_2019.dbf'

In [ ]:
tmc_attr_table_dbf = DBF(tmc_attr_table_fn)
tmc_attr_table_df = pd.DataFrame(iter(tmc_attr_table_dbf))

In [ ]:
# Join DF with TMC-level stats with TMC attribute table
T_df = j4.merge(right=tmc_attr_table_df, left_on='tmc_code', right_on='tmc')

In [ ]:
# Calculate terms used in numerator and denominator of NHPP reliability measure
# Round 'miles' to nearest thousandth, per FHWA doc
T_df['lottr_measure'] = round(T_df['miles'], 3) * T_df['max_lottr'] 

In [ ]:
# Calculate numerator
numerator = T_df['lottr_measure'].sum()
numerator

In [ ]:
# Calculate denominator: sum of 'miles' for all reporting TMCs
denominator = T_df['miles'].sum()
denominator

In [ ]:
Freight_Reliability = numerator / denominator
Freight_Reliability